In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm

In [2]:
def resblock(inputs, num_feature_maps):
    # BLOCK 1

    conv_x = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=3, padding='same')(inputs)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=3, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)


    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=1, padding='same')(inputs)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    outputs = keras.layers.add([shortcut_y, conv_y])
    outputs = keras.layers.Activation('relu')(outputs)
    return outputs

In [3]:
def build_model(input_shape, batch_size):
    inputs = keras.layers.Input(shape=input_shape, batch_size=32)

    layer1 = resblock(inputs, 64)
    layer1 = resblock(layer1, 64)

    layer2 = resblock(layer1, 128)
    layer2 = resblock(layer2, 128)

    layer3 = resblock(layer1, 256)
    layer3 = resblock(layer2, 256)

    layer3 = resblock(layer1, 512)
    layer3 = resblock(layer2, 512)

    pool = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(layer3)

    output = keras.layers.Dense(5, activation='softmax')(pool)



    return keras.models.Model(inputs=inputs, outputs=output)

In [4]:
model = build_model((137, 15), 32)

In [5]:
# model.summary()

In [6]:
f = np.load("mfcc.npz")
X, Y = f['X'], f['Y']
x_train =  X[100:1000]
y_train =  Y[100:1000]


x_test, y_test = X[0:100], Y[0:100]

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train[0:800], y_train[0:800]))
train_dataset = train_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(32, drop_remainder=True)
x_val = x_train[800:]
y_val = y_train[800:]

In [8]:
# optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, weight_decay=0.0001)
# model.compile(optimizer=optimizer, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [9]:
# model.fit(x_train[], y_train, batch_size=32, epochs=2000, validation_data=(x_test, y_test))


In [10]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
acc_metric = keras.metrics.SparseCategoricalAccuracy()
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, weight_decay=0.0001)


@tf.function
def training_step(x, y):

    with tf.GradientTape() as model_tape:
        logits = model(x, training=True)
        loss = loss_fn(y, logits)
    grads = model_tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    acc_metric.update_state(y, logits)
    acc = acc_metric.result()
    acc_metric.reset_states()

    return loss, acc

log = {"training_loss":[], "training_acc":[],
        "val_loss":[], "val_acc":[], "test_acc":[], "test_logits":[]}
log_path = "log" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".npy"
for epoch in tqdm(range(2000)):
    epoch_loss = 0
    epoch_acc = 0
    for step, (x, y) in enumerate(train_dataset):
        batch_loss, batch_acc = training_step(x, y)
        epoch_loss += float(batch_loss)
        epoch_acc += float(batch_acc)
    epoch_loss /= step+1
    epoch_acc /= step+1

    val_loss = 0
    val_acc = 0
    val_logits = model(x_val, training=False)
    val_loss = loss_fn(y_val, val_logits)
    acc_metric.update_state(y_val, val_logits)
    val_acc = acc_metric.result().numpy()
    acc_metric.reset_states()

    # print("Validation loss: %.4f" % (float(val_loss)))
    # print("Validation acc: %.4f" % (float(val_acc)))

    test_logits = model(x_test, training=False)
    acc_metric.update_state(y_test, test_logits)
    test_acc = acc_metric.result().numpy()
    acc_metric.reset_states()

    log["training_loss"].append(epoch_loss)
    log["training_acc"].append(epoch_acc)
    log["val_loss"].append(val_loss)
    log["val_acc"].append(val_acc)
    log["test_acc"].append(test_acc)
    log["test_logits"].append(test_logits)


    np.save(log_path, [log])

log['test_acc'] = np.array(log['test_acc'])
log['val_loss'] = np.array(log['val_loss'])
testing_metric = 0
if len(log['test_acc'][np.where(log['val_loss']-min(log['val_loss'])<1e-6)]) != 0:
    testing_metric = log['test_acc'][np.where((max(log['val_acc']-log['val_loss']) - (log['val_acc']-log['val_loss']))<1e-6)][0]
print(testing_metric)

100%|██████████| 2000/2000 [26:03<00:00,  1.28it/s]

0.98
